# Lab 1 - A*

An implementation of the A* search algorithm ideated and programmed by **Claudio Savelli (s)** & **Mattia Sabato (s305849)** during a cold and rainy October. We have designed the algorithm in order to not only reach the goal in the **fastest way** but also in a **smart way**. We achieve this by reducing the number of overlappings and by forcing a solution which has the less number of shared points in the sets considered.

In [75]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from time import time

import numpy as np

In [76]:
# Set the configuration of the problem.
PROBLEM_SIZE = 9    # Number of points to be covered.
NUM_SETS = 40       # Number of sets in our pool.
ALPHA = 0.3         # Probability for a set to cover one point.

State = namedtuple('State', ['taken', 'not_taken'])

SETS = tuple(np.array([random() < ALPHA for _ in range(PROBLEM_SIZE)]) \
                for _ in range (NUM_SETS))


In [77]:
def goal_check(state: namedtuple) -> bool:
    '''
    Checks whether the problem is solvable and return `False` if not.
    '''
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], \
                         np.array([False for _ in range(PROBLEM_SIZE)])))


def proximity(state: namedtuple) -> int:
    '''
    Computes the proximity between a candidate state and the goal by
    summing the updated number of covered points.
    '''
    return sum(reduce(np.logical_or, [SETS[i] for i in state.taken],
               np.array([False for _ in range(PROBLEM_SIZE)])))


def cost(state) -> float:
    '''
    Computes the cost associated to a candidate state by summing the
    overlappings over the same point.
    '''
    if len(state.taken) == 0:
        return 0
    else:
        return sum(np.array([SETS[i] for i in state.taken]).sum(axis = 0))

assert goal_check(State(set(range(NUM_SETS)), set())), 'Problem not solvable'

In [78]:
frontier = PriorityQueue() 
# frontier = SimpleQueue()
# frontier = LifoQueue()

state = State(set(), set(range(NUM_SETS)))
frontier.put((proximity(state), state))

counter = 0
_, current_state = frontier.get()

tic = time()
while not goal_check(current_state): 

    counter += 1

    for action in current_state.not_taken:

        new_state = State(current_state.taken ^ {action}, \
                          current_state.not_taken ^ {action})
        frontier.put((cost(new_state) - proximity(new_state), new_state))
    
    _, current_state = frontier.get()
    
toc = time()

print(f'Solved in {counter:,} steps ({len(current_state.taken)} tiles) \
in {(toc - tic)*1000:.0f}ms')
print(current_state)

Solved in 486 steps (3 tiles) in 594ms
State(taken={18, 35, 12}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39})
